In [15]:
import pandas as pd
import numpy as np

## load the pedigree code

In [16]:
# Import the pedigree data

"""
Author: Tilman.Ronneburg@imbim.uu.se
20190711
"""
import pandas as pd
import numpy as np
import os
import copy
import scipy
from operator import itemgetter


class Pedigree(object):
    """ Object for parsing pedigree information.
        Requires pedigree file with tab separated fields for:
        sample_id, sire, dam and sex in that order. """
    def __init__(self,pedigree_file):
        self.pedigree = self.load_pedigree(pedigree_file)


    def load_pedigree(self, pedfile):
        """Returns a dictionary from a input file """
        raw_ped = pd.read_csv(pedfile, sep="\t")
        raw_ped.index=raw_ped["ID"]
        ped_d = {}
        for i, k in raw_ped.iterrows():
            ped_d[str(i)] = {"sire":str(k["Sire"]).split(".")[0],
                        "dam":str(k["Dam"]).split(".")[0],
                        "sex":str(k["Sex"]),
                        "generation":str(i)[-2:]}
        return ped_d


    def _old_load_pedigree(self, pedfile):
        """Returns a dictionary from a input file """
        raw_ped = open(pedfile).read().split("\n")
        ped_d = {}
        for line in raw_ped:
            if not line=="":
                l = line.split("\t")
                ped_d[l[0]]={"sire":str(l[1]),
                             "dam":str(l[2]),
                             "sex":str(l[3]),
                             "generation":str(l[0][-2:]),}
        return ped_d


    def get_parents(self,sample):
        """ Returns dam and sire for the the selected sample."""
        dam, sire = self.pedigree[sample]["dam"],self.pedigree[sample]["sire"]
        return dam,sire


    def get_offspring(self, sample):
        """Returns all offspring for the the selected sample."""
        offspring_list = []
        for key, item in self.pedigree.items():
            if sample in [item["sire"], item["dam"]]:
                offspring_list.append(key)
        return offspring_list


    def get_siblings(self,sample, half=False):#
        """ Returns all siblings for the the selected sample.
            If half=True, halfsibs are also returned."""
        dam, sire = self.get_parents(sample=sample)#
        sibs = []
        for key, item in self.pedigree.items():
            if half==False:
                if dam ==item["dam"] and sire==item["sire"]:
                    sibs.append(key)
            if half==True:
                if dam ==item["dam"] or sire==item["sire"]:
                    sibs.append(key)
        return sibs


    def get_families(self, generation,min_size_threshold=5):
        """ Returns a list of lists, where each list represents all individuals
            in the chosen generation that share both parents,
            if the minimum threshold for family size is reached."""
        families = []
        assigned_individuals = []
        for key, item in self.pedigree.items():
            ## lstrip() to deal with inconsistent 03/3 naming of generations
            if item["generation"].lstrip("0")==str(generation).lstrip("0"):
                sibs = self.get_siblings(sample=key, half=False)
                if not key in assigned_individuals:
                    assigned_individuals= assigned_individuals + sibs
                    if len(sibs)>=min_size_threshold:
                        families.append(sibs)
        return families


    def write_pedigree(self, filename):
        """ Writes the pedigree out to new file. """
        with open(filename, "w") as handle:
            handle.write("ID\tsire\tdam\tsex\n")
            for key, item in self.pedigree.items():
                handle.write("\t".join([key,
                                        item["sire"],
                                        item["dam"],
                                        item["sex"]])+"\n")





### load the samples i need to look at ( its hacky, i know)

In [17]:
gt_ids = [
"44902" ,
"51202" ,
"120502" ,
"48402" ,
"41402" ,
"130502" ,
"59402" ,
"105602" ,
"71702" ,
"96702" ,
"141002" ,
"43002" ,
"117502" ,
"102902" ,
"100902" ,
"52802" ,
"109902" ,
"58802" ,
"53702" ,
"37902" ,
"106702" ,
"76602" ,
"68402" ,
"131202" ,
"104302" ,
"129202" ,
"145502" ,
"127202" ,
"104802" ,
"103002" ,
"50002" ,
"61702" ,
"45102" ,
"94002" ,
"71202" ,
"108902" ,
"142402" ,
"70102" ,
"70402" ,
"50502" ,
"80502" ,
"104002" ,
"104402" ,
"131402" ,
"81402" ,
"62602" ,
"61102" ,
"74102" ,
"81602" ,
"124402" ,
"108702" ,
"38002" ,
"98802" ,
"115302" ,
"99702" ,
"39202" ,
"119002" ,
"93202" ,
"116102" ,
"56602" ,
"122802" ,
"127702" ,
"129802" ,
"98202" ,
"114002" ,
"67702" ,
"82502" ,
"68702" ,
"131602" ,
"106202" ,
"140602" ,
"110902" ,
"85002" ,
"35502" ,
"97602" ,
"91702" ,
"78602" ,
"100802" ,
"126202" ,
"118102" ,
"63002" ,
"108502" ,
"56802" ,
"110602" ,
"82702" ,
"85102" ,
"69502" ,
"69602" ,
"117002" ,
"105202" ,
"113902" ,
"58902" ,
"55202" ,
"35102" ,
"40702" ,
"44002" ,
"48302" ,
"72402" ,
"132202" ,
"69802" ,
"49202" ,
"139202" ,
"93602" ,
"69302" ,
"80202" ,
"60302" ,
"94502" ,
"85502" ,
"57702" ,
"114302" ,
"118302" ,
"128402" ,
"118702" ,
"85402" ,
"139002" ,
"88702" ,
"73602" ,
"139102" ,
"94802" ,
"39902" ,
"83302" ,
"66302" ,
"125802" ,
"112202" ,
"80302" ,
"103902" ,
"96102" ,
"99502" ,
"115202" ,
"42802" ,
"66602" ,
"101302" ,
"98302" ,
"96202" ,
"95802" ,
"139702" ,
"65102" ,
"64402" ,
"41702" ,
"117102" ,
"101702" ,
"68602" ,
"143202" ,
"68902" ,
"85602" ,
"110502" ,
"56102" ,
"105502" ,
"37202" ,
"50202" ,
"57902" ,
"124102" ,
"56502" ,
"72602" ,
"94302" ,
"95702" ,
"101502" ,
"81702" ,
"67602" ,
"99602" ,
"146902" ,
"62902" ,
"74202" ,
"104602" ,
"60002" ,
"125502" ,
"64202" ,
"72202" ,
"135102" ,
"84902" ,
"77202" ,
"143002" ,
"142002" ,
"114902" ,
"67102" ,
"39002" ,
"105102" ,
"74902" ,
"80102" ,
"130802" ,
"52502" ,
"100002" ,
"108402" ,
"120602" ,
"107102" ,
"109502" ,
"38302" ,
"86202" ,
"44102" ,
"93502" ,
"138402" ,
"63802" ,
"54802" ,
"122402" ,
"48902" ,
"121202" ,
"111602" ,
"52002" ,
"76502" ,
"137402" ,
"71902" ,
"58202" ,
"37102" ,
"52902" ,
"40002" ,
"118902" ,
"68802" ,
"87402" ,
"135002" ,
"74302" ,
"87802" ,
"59302" ,
"119102" ,
"68002" ,
"136302" ,
"132802" ,
"138302" ,
"88802" ,
"90802" ,
"113402" ,
"100602" ,
"39802" ,
"102402" ,
"65902" ,
"143402" ,
"141902" ,
"72902" ,
"134402" ,
"107202" ,
"137702" ,
"84202" ,
"83502" ,
"116902" ,
"129702" ,
"63902" ,
"35202" ,
"67502" ,
"132902" ,
"41002" ,
"131302" ,
"142102" ,
"38602" ,
"78402" ,
"60402" ,
"115602" ,
"135402" ,
"118002" ,
"143602" ,
"122202" ,
"98902" ,
"73302" ,
"109602" ,
"44202" ,
"35902" ,
"102302" ,
"107802" ,
"56402" ,
"60602" ,
"41302" ,
"117902" ,
"86602" ,
"35002" ,
"63302" ,
"58502" ,
"53202" ,
"76402" ,
"72302" ,
"88402" ,
"116402" ,
"103402" ,
"44602" ,
"74802" ,
"112502" ,
"59802" ,
"118502" ,
"46102" ,
"98402" ,
"130602" ,
"123302" ,
"79702" ,
"56902" ,
"90002" ,
"77602" ,
"108302" ,
"41102" ,
"72802" ,
"81102" ,
"57102" ,
"36802" ,
"36402" ,
"91302" ,
"75002" ,
"65502" ,
"70302" ,
"130902" ,
"73902" ,
"66202" ,
"116702" ,
"122702" ,
"79902" ,
"64002" ,
"119602" ,
"77502" ,
"44502" ,
"93802" ,
"53602" ,
"84302" ,
"103102" ,
"50102" ,
"133902" ,
"135202" ,
"81902" ,
"76202" ,
"110302" ,
"54602" ,
"60502" ,
"40502" ,
"122902" ,
"54902" ,
"109402" ,
"85702" ,
"84502" ,
"50402" ,
"119902" ,
"138702" ,
"68202" ,
"80402" ,
"55402" ,
"78202" ,
"78902" ,
"83602" ,
"79402" ,
"96602" ,
"67402" ,
"120202" ,
"70002" ,
"97502" ,
"100402" ,
"141602" ,
"102802" ,
"109002" ,
"87002" ,
"120702" ,
"69102" ,
"46702" ,
"89502" ,
"129002" ,
"60902" ,
"136902" ,
"109802" ,
"111702" ,
"44302" ,
"37302" ,
"54202" ,
"123602" ,
"61402" ,
"51402" ,
"100202" ,
"59002" ,
"96302" ,
"51002" ,
"119702" ,
"50702" ,
"96902" ,
"129402" ,
"113102" ,
"78302" ,
"91202" ,
"106002" ,
"47002" ,
"102002" ,
"82802" ,
"38802" ,
"136702" ,
"118802" ,
"82902" ,
"88502" ,
"125302" ,
"82302" ,
"79802" ,
"45602" ,
"93102" ,
"132102" ,
"79502" ,
"110402" ,
"127802" ,
"127002" ,
"134202" ,
"53402" ,
"81002" ,
"62502" ,
"63502" ,
"48602" ,
"76302" ,
"80002" ,
"72002" ,
"75402" ,
"108002" ,
"120802" ,
"78502" ,
"111202" ,
"119202" ,
"60802" ,
"55002" ,
"62202" ,
"146302" ,
"87502" ,
"65702" ,
"67802" ,
"68102" ,
"68502" ,
"70902" ,
"71002" ,
"105002" ,
"69702" ,
"37402" ,
"134002" ,
"78802" ,
"73402" ,
"89102" ,
"110202" ,
"39302" ,
"128702" ,
"135902" ,
"138602" ,
"111802" ,
"91402" ,
"80702" ,
"53902" ,
"67202" ,
"71502" ,
"89402" ,
"76902" ,
"77102" ,
"86902" ,
"46802" ,
"106102" ,
"126902" ,
"80802" ,
"126502" ,
"130402" ,
"90902" ,
"56202" ,
"142202" ,
"118202" ,
"123702" ,
"85902" ,
"75102" ,
"63602" ,
"97002" ,
"121702" ,
"58002" ,
"48202" ,
"40402" ,
"44702" ,
"46202" ,
"84602" ,
"49702" ,
"108202" ,
"88902" ,
"99902" ,
"91502" ,
"101402" ,
"140802" ,
"103602" ,
"95902" ,
"73702" ,
"92002" ,
"105802" ,
"111302" ,
"144502" ,
"62102" ,
"100302" ,
"38102" ,
"114602" ,
"124802" ,
"52602" ,
"89202" ,
"99802" ,
"55802" ,
"136102" ,
"68302" ,
"142802" ,
"113802" ,
"95002" ,
"58602" ,
"38702" ,
"62402" ,
"44802" ,
"42902" ,
"134702" ,
"141402" ,
"124202" ,
"140402" ,
"55502" ,
"79602" ,
"92302" ,
"95402" ,
"90202" ,
"96802" ,
"100102" ,
"87202" ,
"57602" ,
"94202" ,
"83702" ,
"57002" ,
"36002" ,
"59202" ,
"111902" ,
"83002" ,
"79002" ,
"140302" ,
"107702" ,
"116602" ,
"128202" ,
"73502" ,
"115102" ,
"117802" ,
"110102" ,
"67302" ,
"49802" ,
"104502" ,
"83202" ,
"55602" ,
"59502" ,
"129602" ,
"101802" ,
"124502" ,
"73002" ,
"107402" ,
"131802" ,
"112702" ,
"90102" ,
"81202" ,
"56702" ,
"63402" ,
"37602" ,
"128602" ,
"112102" ,
"98002" ,
"37002" ,
"117402" ,
"83102" ,
"139902" ,
"88002" ,
"128302" ,
"138902" ,
"41802" ,
"112802" ,
"135602" ,
"45302" ,
"35302" ,
"61902" ,
"39502" ,
"63702" ,
"130202" ,
"76702" ,
"99202" ,
"135502" ,
"50802" ,
"71802" ,
"66902" ,
"52202" ,
"98702" ,
"38902" ,
"134902" ,
"56002" ,
"78102" ,
"47402" ,
"131102" ,
"41202" ,
"125102" ,
"122502" ,
"99002" ,
"138102" ,
"52102" ,
"58702" ,
"43902" ,
"126402" ,
"141502" ,
"42502" ,
"84402" ,
"112302" ,
"57802" ,
"128002" ,
"105902" ,
"139302" ,
"35702" ,
"97902" ,
"45502" ,
"62802" ,
"144302" ,
"122102" ,
"143702" ,
"94102" ,
"50302" ,
"115502" ,
"64502" ,
"35602" ,
"135302" ,
"84102" ,
"95202" ,
"71102" ,
"107502" ,
"103802" ,
"70202" ,
"36602" ,
"84702" ,
"83802" ,
"59902" ,
"109102" ,
"111502" ,
"106802" ,
"91902" ,
"79202" ,
"82002" ,
"107902" ,
"124702" ,
"47202" ,
"99302" ,
"75202" ,
"41902" ,
"77902" ,
"85302" ,
"103302" ,
"104702" ,
"77702" ,
"36702" ,
"86702" ,
"42702" ,
"86102" ,
"99402" ,
"53302" ,
"120302" ,
"64102" ,
"126702" ,
"124602" ,
"36202" ,
"92702" ,
"105702" ,
"102702" ,
"121002" ,
"69002" ,
"64902" ,
"61602" ,
"50602" ,
"123902" ,
"102602" ,
"114802" ,
"114202" ,
"102502" ,
"132302" ,
"124902" ,
"86502" ,
"121302" ,
"76002" ,
"40602" ,
"95302" ,
"87702" ,
"114702" ,
"78702" ,
"145302" ,
"96002" ,
"87102" ,
"125002" ,
"56302" ,
"130302" ,
"111102" ,
"65602" ,
"127302" ,
"133302" ,
"142602" ,
"105302" ,
"45202" ,
"103502" ,
"64302" ,
"93702" ,
"113002" ,
"88602" ,
"51302" ,
"61502" ,
"95102" ,
"52702" ,
"53002" ,
"51602" ,
"109702" ,
"64702" ,
"140002" ,
"128902" ,
"42102" ,
"38402" ,
"92202" ,
"106602" ,
"77002" ,
"79302" ,
"146402" ,
"39102" ,
"131702" ,
"138202" ,
"66502" ,
"97802" ,
"42202" ,
"126602" ,
"71302" ,
"90302" ,
"136402" ,
"77302" ,
"103202" ,
"89802" ,
"115402" ,
"50902" ,
"134102" ,
"121802" ,
"58302" ,
"97202" ,
"40202" ,
"80902" ,
"35402" ,
"52302" ,
"119802" ,
"51702" ,
"49302" ,
"131502" ,
"54102" ,
"116202" ,
"79102" ,
"142502" ,
"110702" ,
"55902" ,
"101202" ,
"104202" ,
"133602" ,
"75902" ,
"120402" ,
"82402" ,
"125202" ,
"94602" ,
"86302" ,
"142902" ,
"104102" ,
"137802" ,
"88102" ,
"97702" ,
"120102" ,
"142302" ,
"94402" ,
"39402" ,
"40302" ,
"69202" ,
"116302" ,
"62002" ,
"90502" ,
"119502" ,
"63102" ,
"111402" ,
"137302" ,
"86402" ,
"66802" ,
"55702" ,
"100502" ,
"53802" ,
"96402" ,
"36502" ,
"93302" ,
"134302" ,
"141302" ,
"61202" ,
"59102" ,
"137902" ,
"70702" ,
"115702" ,
"145202" ,
"54502" ,
"86802" ,
"53102" ,
"59602" ,
"133102" ,
"44402" ,
"83402" ,
"65002" ,
"52402" ,
"98602" ,
"114402" ,
"117702" ,
"121602" ,
"36102" ,
"94702" ,
"116502" ,
"115902" ,
"141202" ,
"36902" ,
"125402" ,
"99102" ,
"112002" ,
"88302" ,
"69902" ,
"70602" ,
"145402" ,
"60102" ,
"89902" ,
"65202" ,
"74502" ,
"90702" ,
"59702" ,
"46902" ,
"72102" ,
"94902" ,
"91602" ,
"120002" ,
"112902" ,
"85202" ,
"101602" ,
"75602" ,
"51102" ,
"130002" ,
"98102" ,
"37802" ,
"66402" ,
"64802" ,
"119402" ,
"72502" ,
"35802" ,
"122002" ,
"49502" ,
"125902" ,
"126002" ,
"104902" ,
"65402" ,
"61302" ,
"114102" ,
"140702" ,
"93402" ,
"39602" ,
"87602" ,
"60202" ,
"129502" ,
"123802" ,
"54702" ,
"80602" ,
"85802" ,
"140202" ,
"117302" ,
"122302" ,
"119302" ,
"103702" ,
"100702" ,
"112602" ,
"66002" ,
"107002" ,
"98502" ,
"128802" ,
"129102" ,
"75302" ,
"38502" ,
"51502" ,
"109302" ,
"132502" ,
"81502" ,
"133202" ,
"97302" ,
"147002" ,
"67902" ,
"41602" ,
"127102" ,
"117202" ,
"106502" ,
"122602" ,
"54302" ,
"40802" ,
"109202" ,
"46302" ,
"138002" ,
"74602" ,
"128102" ,
"116802" ,
"110802" ,
"112402" ,
"36302" ,
"106402" ,
"101002" ,
"40902" ,
"61802" ,
"92902" ,
"65802" ,
"57502" ,
"58402" ,
"105402" ,
"49102" ,
"123102" ,
"137102" ,
"118402" ,
"144902" ,
"127502" ,
"107302" ,
"106902" ,
"129302" ,
"143502" ,
"73802" ,
"108102" ,
"123502" ,
"96502" ,
"47902" ,
"91802" ,
"62302" ,
"77402" ,
"106302" ,
"147102" ,
"48502" ,
"146802" ,
"57402" ,
"130702" ,
"102202" ,
"97402" ,
"69402" ,
"140102" ,
"111002" ,
"51902" ,
"62702" ,
"87902" ,
"93002" ,
"61002" ,
"127402" ,
"74002" ,
"42402" ,
"121502" ,
"92602" ,
"121902" ,
"71402" ,
"66102" ,
"89002" ,
"339704" ,
"355204" ,
"355804" ,
"352704" ,
"344604" ,
"340804" ,
"376704" ,
"345704" ,
"341304" ,
"336804" ,
"375604" ,
"377804" ,
"373204" ,
"352604" ,
"365904" ,
"358304" ,
"364604" ,
"332904" ,
"326304" ,
"370304" ,
"363804" ,
"334004" ,
"357604" ,
"367804" ,
"345404" ,
"342204" ,
"335204" ,
"333704" ,
"367104" ,
"345004" ,
"327104" ,
"366404" ,
"354604" ,
"359304" ,
"378804" ,
"331504" ,
"347604" ,
"343704" ,
"341204" ,
"372504" ,
"331004" ,
"371804" ,
"358604" ,
"343404" ,
"331804" ,
"351604" ,
"347204" ,
"330604" ,
"350604" ,
"345104" ,
"333304" ,
"377604" ,
"362504" ,
"361804" ,
"334104" ,
"327704" ,
"330304" ,
"368704" ,
"338104" ,
"340504" ,
"365704" ,
"344704" ,
"354504" ,
"331704" ,
"368004" ,
"356504" ,
"347904" ,
"338304" ,
"356004" ,
"358504" ,
"374304" ,
"346204" ,
"364704" ,
"329604" ,
"326204" ,
"337004" ,
"368804" ,
"359204" ,
"366304" ,
"342704" ,
"330904" ,
"339304" ,
"343504" ,
"361304" ,
"332804" ,
"373604" ,
"346704" ,
"335404" ,
"376804" ,
"347504" ,
"361204" ,
"357904" ,
"327004" ,
"337104" ,
"355004" ,
"343604" ,
"370704" ,
"374104" ,
"348204" ,
"353804" ,
"375704" ,
"360504" ,
"367704" ,
"364204" ,
"339204" ,
"349504" ,
"372004" ,
"327904" ,
"339604" ,
"328904" ,
"351804" ,
"344404" ,
"857306" ,
"841106" ,
"866506" ,
"860306" ,
"865306" ,
"861206" ,
"853706" ,
"854406" ,
"840106" ,
"864306" ,
"856506" ,
"853206" ,
"855306" ,
"854906" ,
"852506" ,
"848406" ,
"855806" ,
"855006" ,
"862406" ,
"842106" ,
"864206" ,
"840906" ,
"856806" ,
"861906" ,
"846206" ,
"865206" ,
"844806" ,
"851006" ,
"847006" ,
"866106" ,
"862806" ,
"858206" ,
"857206" ,
"846906" ,
"847306" ,
"848806" ,
"863706" ,
"847806" ,
"844406" ,
"862906" ,
"846306" ,
"851806" ,
"851106" ,
"858906" ,
"859506" ,
"852106" ,
"845006" ,
"843806" ,
"842506" ,
"843306" ,
"849406" ,
"864506" ,
"856706" ,
"844106" ,
"840206" ,
"849706" ,
"847406" ,
"861706" ,
"848706" ,
"846006" ,
"861006" ,
"863606" ,
"853106" ,
"855406" ,
"841906" ,
"863206" ,
"865106" ,
"860706" ,
"859606" ,
"851406" ,
"843406" ,
"847506" ,
"844906" ,
"859906" ,
"847906" ,
"852906" ,
"863306" ,
"858806" ,
"860006" ,
"841306" ,
"854606" ,
"840506" ,
"841706" ,
"843706" ,
"845506" ,
"858406" ,
"854506" ,
"863806" ,
"438310" ,
"377710" ,
"375710" ,
"377110" ,
"438110" ,
"384510" ,
"430410" ,
"387110" ,
"427410" ,
"427210" ,
"375310" ,
"429710" ,
"379410" ,
"375610" ,
"385710" ,
"431410" ,
"373210" ,
"385610" ,
"382910" ,
"373510" ,
"387010" ,
"384410" ,
"384810" ,
"374510" ,
"386510" ,
"382610" ,
"377510" ,
"379810" ,
"382210" ,
"431510" ,
"373910" ,
"431110" ,
"383910" ,
"381510" ,
"386910" ,
"384910" ,
"386210" ,
"384710" ,
"386310" ,
"382110" ,
"426510" ,
"385810" ,
"373710" ,
"377010" ,
"377210" ,
"382710" ,
"384310" ,
"379210" ,
"376910" ,
"376210" ,
"378510" ,
"386810" ,
"434810" ,
"374010" ,
"381610" ,
"381410" ,
"375110" ,
"434310" ,
"380310" ,
"376010" ,
"549908" ,
"541808" ,
"519008" ,
"553808" ,
"520108" ,
"520808" ,
"517508" ,
"560708" ,
"558008" ,
"531508" ,
"541608" ,
"554208" ,
"561508" ,
"530108" ,
"558608" ,
"558408" ,
"531108" ,
"554908" ,
"540608" ,
"561608" ,
"560008" ,
"529908" ,
"540108" ,
"521708" ,
"550908" ,
"539608" ,
"555208" ,
"561708" ,
"559008" ,
"558908" ,
"554808" ,
"551108" ,
"528908" ,
"556308" ,
"553108" ,
"540308" ,
"550208" ,
"555108" ,
"561808" ,
"518008" ,
"532508" ,
"555708" ,
"520408" ,
"517808" ,
"542408" ,
"555608" ,
"561908" ,
"540908" ,
"528408" ,
"517108" ,
"550708" ,
"534708" ,
"554708" ,
"521608" ,
"518408" ,
"549808" ,
"534108" ,
"542608" ,
"539308" ,
"556808" ,
"533108" ,
"528808" ,
"539708" ,
"557508" ,
"518908" ,
"556408" ,
"550508" ,
"532108" ,
"516808" ,
"551908" ,
"555808" ,
"559408" ,
"558208" ,
"554308" ,
"529608" ,
"550308" ,
"556708" ,
"517408" ,
"551708" ,
"559508" ,
"551508" ,
"561008" ,
"528208" ,
"521808" ,
"532208" ,
"548208" ,
"527708" ,
"540208" ,
"557108" ,
"528508" ,
"519808" ,
"559308" ,
"530508" ,
"548808" ,
"555408" ,
"532408" ,
"553408" ,
"520208" ,
"547208" ,
"553208" ,
"540808" ,
"561208" ,
"542008" ,
"548308" ,
"552308" ,
"516708" ,
"558308" ,
"539808" ,
"547708" ,
"532008" ,
"520608" ,
"554608" ,
"554408" ,
"547808" ,
"529308" ,
"549008" ,
"541308" ,
"556908" ,
"541508" ,
"529408" ,
"546808" ,
"534808" ,
"528708" ,
"542108" ,
"531808" ,
"540508" ,
"547108" ,
"540008" ,
"547008" ,
"533208" ,
"539908" ,
"531208" ,
"539508" ,
"521508" ,
"531708" ,
"539108" ,
"550008" ,
"549208" ,
"557908" ,
"521908" ,
"530408" ,
"531908" ,
"530008" ,
"555908" ,
"552808" ,
"549508" ,
"555508" ,
"533808" ,
"552108" ,
"522108" ,
"558808" ,
"522208" ,
"527908" ,
"548108" ,
"542708" ,
"532608" ,
"559808" ,
"553708" ,
"539408" ,
"549308" ,
"534208" ,
"557408" ,
"556008" ,
"555008" ,
"540408" ,
"551308" ,
"562208" ,
"558508" ,
"542808" ,
"521108" ,
"519908" ,
"533408" ,
"550608" ,
"534608" ,
"520708" ,
"540708" ,
"517208" ,
"519208" ,
"550408" ,
"549408" ,
"547508" ,
"551008" ,
"529508" ,
"529108" ,
"529008" ,
"522308" ,
"553308" ,
"552208" ,
"541008" ,
"531308" ,
"528308" ,
"557008" ,
"542508" ,
"559208" ,
"553008" ,
"552908" ,
"530708" ,
"556508" ,
"548708" ,
"556208" ,
"560508" ,
"533708" ,
"561108" ,
"528608" ,
"559708" ,
"532308" ,
"550108" ,
"518808" ,
"555308" ,
"552008" ,
"558708" ,
"559908" ,
"554508" ,
"530808" ,
"547608" ,
"542308" ,
"541708" ,
"520308" ,
"519108" ,
"518308" ,
"518208" ,
"530908" ,
"548608" ,
"529708" ,
"552608" ,
"533508" ,
"531008" ,
"516908" ,
"527808" ,
"518708" ,
"554108" ,
"547408" ,
"562408" ,
"518508" ,
"557208" ,
"517708" ,
"541108" ,
"520908" ,
"541408" ,
"557308" ,
"558108" ,
"531408" ,
"559608" ,
"519708" ,
"560808" ,
"560208" ,
"549108" ,
"534408" ,
"533608" ,
"560408" ,
"517008" ,
"517908" ,
"548408" ,
"534308" ,
"546908" ,
"560108" ,
"556608" ,
"527608" ,
"533908" ,
"550808" ,
"552708" ,
"547308" ,
"554008" ,
"520008" ,
"531608" ,
"551408" ,
"530208" ,
"541908" ,
"556108" ,
"552408" ,
"530608" ,
"521008" ,
"529208" ,
"553908" ,
"534008" ,
"517608" ,
"517308" ,
"519608" ,
"532808" ,
"553508" ,
"549608" ,
"522008" ,
"520508" ,
"557808" ,
"557708" ,
"534508" ,
"542208" ,
"549708" ,
"561408" ,
"528008" ,
"532908" ,
"557608" ,
"290315" ,
"297115" ,
"282615" ,
"298215" ,
"238115" ,
"321615" ,
"302815" ,
"327215" ,
"286815" ,
"310915" ,
"311115" ,
"275215" ,
"280015" ,
"305915" ,
"288115" ,
"246715" ,
"333515" ,
"278915" ,
"308215" ,
"236615" ,
"304015" ,
"325415" ,
"301515" ,
"302315" ,
"313315" ,
"305515" ,
"253815" ,
"271515" ,
"238415" ,
"260215" ,
"309215" ,
"251015" ,
"255215" ,
"273915" ,
"258015" ,
"256415" ,
"244815" ,
"255315" ,
"291615" ,
"248915" ,
"318715" ,
"325515" ,
"235315" ,
"257015" ,
"245615" ,
"296015" ,
"295315" ,
"235015" ,
"283515" ,
"303715" ,
"315615" ,
"237815" ,
"283615" ,
"236315" ,
"260615" ,
"243115" ,
"307115" ,
"302415" ,
"320015" ,
"322315" ,
"271715" ,
"285515" ,
"291715" ,
"318215" ,
"294415" ,
"330415" ,
"253915" ,
"246815" ,
"331415" ,
"299015" ,
"250315" ,
"329515" ,
"237615" ,
"301415" ,
"234215" ,
"325115" ,
"287315" ,
"240415" ,
"314815" ,
"280515" ,
"278115" ,
"311515" ,
"257815" ,
"296415" ,
"246515" ,
"329615" ,
"312515" ,
"295515" ,
"276815" ,
"273415" ,
"302615" ,
"293615" ,
"261915" ,
"302915" ,
"301115" ,
"249215" ,
"310715" ,
"312615" ,
"334515" ,
"247415" ,
"319215" ,
"296215" ,
"241915" ,
"292815" ,
"298115" ,
"313615" ,
"306615" ,
"276215" ,
"308815" ,
"273715" ,
"294915" ,
"252415" ,
"286615" ,
"282715" ,
"311715" ,
"287915" ,
"242415" ,
"319015" ,
"299915" ,
"277915" ,
"251515" ,
"329015" ,
"331315" ,
"256515" ,
"281015" ,
"305715" ,
"272415" ,
"287115" ,
"305215" ,
"272015" ,
"312715" ,
"237215" ,
"293015" ,
"317515" ,
"238015" ,
"242715" ,
"316815" ,
"332715" ,
"260015" ,
"317015" ,
"291515" ,
"245115" ,
"281815" ,
"317815" ,
"295915" ,
"241115" ,
"331915" ,
"251715" ,
"245715" ,
"251215" ,
"317615" ,
"299715" ,
"289015" ,
"244915" ,
"242615" ,
"304615" ,
"304715" ,
"255615" ,
"320515" ,
"239015" ,
"291215" ,
"280715" ,
"245515" ,
"255515" ,
"275315" ,
"235115" ,
"248615" ,
"323415" ,
"309915" ,
"260815" ,
"326915" ,
"296915" ,
"334415" ,
"242215" ,
"286115" ,
"286315" ,
"283915" ,
"309615" ,
"247315" ,
"322715" ,
"312215" ,
"235915" ,
"332415" ,
"326715" ,
"330615" ,
"233015" ,
"300815" ,
"329715" ,
"332915" ,
"261315" ,
"243515" ,
"316315" ,
"234915" ,
"292215" ,
"261515" ,
"254915" ,
"280415" ,
"331015" ,
"330115" ,
"303215" ,
"303515" ,
"294215" ,
"300215" ,
"322415" ,
"300515" ,
"323915" ,
"279015" ,
"239415" ,
"285015" ,
"243015" ,
"306715" ,
"259715" ,
"235715" ,
"248215" ,
"304515" ,
"291815" ,
"272315" ,
"294115" ,
"289915" ,
"276915" ,
"312415" ,
"317315" ,
"329815" ,
"305315" ,
"313915" ,
"293415" ,
"310515" ,
"292115" ,
"287715" ,
"248315" ,
"285215" ,
"278815" ,
"325815" ,
"327015" ,
"279515" ,
"277815" ,
"240315" ,
"307515" ,
"314915" ,
"235515" ,
"332315" ,
"283815" ,
"319115" ,
"254815" ,
"334115" ,
"288415" ,
"302015" ,
"240115" ,
"272715" ,
"251615" ,
"310015" ,
"314615" ,
"323015" ,
"240815" ,
"238315" ,
"234615" ,
"233415" ,
"274815" ,
"280815" ,
"255715" ,
"321115" ,
"306515" ,
"251415" ,
"283715" ,
"308915" ,
"276115" ,
"319515" ,
"326015" ,
"255915" ,
"333415" ,
"311815" ,
"249015" ,
"237515" ,
"248415" ,
"243815" ,
"276615" ,
"307015" ,
"305015" ,
"273815" ,
"279615" ,
"313215" ,
"258715" ,
"252915" ,
"234415" ,
"253615" ,
"258115" ,
"233915" ,
"316215" ,
"247715" ,
"260715" ,
"301315" ,
"299215" ,
"242315" ,
"320915" ,
"334615" ,
"233115" ,
"284915" ,
"279115" ,
"331615" ,
"282515" ,
"271315" ,
"298315" ,
"237915" ,
"297615" ,
"291415" ,
"271215" ,
"318015" ,
"332115" ,
"320115" ,
"296315" ,
"249715" ,
"306115" ,
"296815" ,
"314115" ,
"333015" ,
"315215" ,
"285415" ,
"253315" ,
"280215" ,
"298515" ,
"329415" ,
"317915" ,
"319815" ,
"254015" ,
"275515" ,
"242515" ,
"258415" ,
"274915" ,
"323615" ,
"241315" ,
"321915" ,
"304215" ,
"321215" ,
"333815" ,
"296515" ,
"278715" ,
"246415" ,
"280115" ,
"291915" ,
"254515" ,
"244515" ,
"285615" ,
"251115" ,
"316015" ,
"237015" ,
"292515" ,
"237115" ,
"322115" ,
"254215" ,
"233715" ,
"280615" ,
"290415" ,
"327315" ,
"310415" ,
"282315" ,
"312015" ,
"326315" ,
"290815" ,
"314215" ,
"284115" ,
"236915" ,
"254715" ,
"329315" ,
"238215" ,
"328215" ,
"245015" ,
"289715" ,
"331815" ,
"281315" ,
"244215" ,
"252515" ,
"242115" ,
"299615" ,
"243715" ,
"281915" ,
"327115" ,
"319915" ,
"249315" ,
"327815" ,
"233815" ,
"233515" ,
"284815" ,
"289515" ,
"313115" ,
"312915" ,
"255115" ,
"317115" ,
"277015" ,
"236515" ,
"233315" ,
"247215" ,
"313015" ,
"247115" ,
"240715" ,
"260415" ,
"293915" ,
"259115" ,
"252615" ,
"324915" ,
"241215" ,
"236115" ,
"241815" ,
"324115" ,
"240215" ,
"328815" ,
"257415" ,
"323115" ,
"244715" ,
"293115" ,
"249615" ,
"248115" ,
"314715" ,
"254115" ,
"259015" ,
"289215" ,
"247515" ,
"300415" ,
"293815" ,
"301715" ,
"303815" ,
"260915" ,
"257515" ,
"283015" ,
"277515" ,
"272615" ,
"320215" ,
"331515" ,
"309115" ,
"301815" ,
"260315" ,
"260515" ,
"303615" ,
"246315" ,
"288215" ,
"244415" ,
"273615" ,
"275915" ,
"250215" ,
"323215" ,
"297015" ,
"259915" ,
"249915" ,
"278015" ,
"304415" ,
"282915" ,
"282215" ,
"280315" ,
"324215" ,
"285915" ,
"272915" ,
"261815" ,
"295715" ,
"236715" ,
"326215" ,
"275715" ,
"333715" ,
"290015" ,
"260115" ,
"325715" ,
"245415" ,
"333615" ,
"234315" ,
"234815" ,
"286415" ,
"285815" ,
"308415" ,
"284615" ,
"311615" ,
"284715" ,
"296115" ,
"239615" ,
"235615" ,
"306415" ,
"308515" ,
"309315" ,
"261615" ,
"316115" ,
"274015" ,
"291115" ,
"261015" ,
"294315" ,
"295115" ,
"278215" ,
"300915" ,
"292315" ,
"330215" ,
"258915" ,
"334215" ,
"319415" ,
"325015" ,
"309515" ,
"293715" ,
"279415" ,
"281215" ,
"307415" ,
"324015" ,
"324815" ,
"320615" ,
"322615" ,
"296615" ,
"298815" ,
"284015" ,
"316915" ,
"310615" ,
"259615" ,
"307215" ,
"239315" ,
"303015" ,
"327915" ,
"308715" ,
"294015" ,
"273515" ,
"317715" ,
"234115" ,
"276315" ,
"246915" ,
"309015" ,
"234015" ,
"271815" ,
"288915" ,
"305815" ,
"241515" ,
"288715" ,
"319315" ,
"274715" ,
"315715" ,
"256915" ,
"238615" ,
"323515" ,
"295215" ,
"250915" ,
"328515" ,
"320315" ,
"318415" ,
"313715" ,
"245215" ,
"334715" ,
"233615" ,
"307715" ,
"324415" ,
"327515" ,
"261715" ,
"290215" ,
"314415" ,
"282015" ,
"322815" ,
"328915" ,
"257215" ,
"272815" ,
"252815" ,
"257615" ,
"243315" ,
"274515" ,
"281615" ,
"261115" ,
"254615" ,
"256015" ,
"256715" ,
"333915" ,
"299415" ,
"272215" ,
"326415" ,
"283215" ,
"242815" ,
"328015" ,
"279815" ,
"332515" ,
"315915" ,
"244315" ,
"254415" ,
"243415" ,
"274315" ,
"325915" ,
"256615" ,
"288515" ,
"293215" ,
"250815" ,
"296715" ,
"299815" ,
"303115" ,
"333115" ,
"239215" ,
"327715" ,
"239815" ,
"316615" ,
"247915" ,
"298415" ,
"250615" ,
"287015" ,
"314015" ,
"246115" ,
"302515" ,
"315515" ,
"243215" ,
"246615" ,
"331215" ,
"315415" ,
"326115" ,
"248015" ,
"238915" ,
"297415" ,
"321015" ,
"287415" ,
"295615" ,
"297215" ,
"302215" ,
"238715" ,
"312315" ,
"275815" ,
"273315" ,
"259815" ,
"256215" ,
"253515" ,
"276415" ,
"287215" ,
"324615" ,
"271615" ,
"241715" ,
"328315" ,
"240015" ,
"238515" ,
"256815" ,
"305115" ,
"320715" ,
"307915" ,
"311415" ,
"297315" ,
"331115" ,
"301915" ,
"332015" ,
"277715" ,
"244615" ,
"330715" ,
"252715" ,
"326815" ,
"282115" ,
"239715" ,
"321715" ,
"284415" ,
"294715" ,
"258215" ,
"298715" ,
"259315" ,
"245815" ,
"284315" ,
"295415" ,
"252015" ,
"249815" ,
"302715" ,
"275415" ,
"306815" ,
"289115" ,
"316515" ,
"303915" ,
"330015" ,
"287815" ,
"305415" ,
"237715" ,
"282815" ,
"258315" ,
"332615" ,
"331715" ,
"298015" ,
"309415" ,
"308115" ,
"289315" ,
"271115" ,
"314315" ,
"274215" ,
"280915" ,
"246215" ,
"255815" ,
"242915" ,
"253115" ,
"252115" ,
"327615" ,
"249415" ,
"240915" ,
"236015" ,
"300715" ,
"239115" ,
"298915" ,
"293515" ,
"278515" ,
"299515" ,
"258515" ,
"272115" ,
"328415" ,
"239915" ,
"301215" ,
"277315" ,
"292015" ,
"288615" ,
"237315" ,
"249515" ,
"311915" ,
"259415" ,
"276015" ,
"312115" ,
"285115" ,
"320815" ,
"330815" ,
"276515" ,
"311215" ,
"288815" ,
"235215" ,
"328115" ,
"309715" ,
"318315" ,
"275615" ,
"247815" ,
"306015" ,
"249115" ,
"292715" ,
"295815" ,
"286215" ,
"273115" ,
"290515" ,
"288015" ,
"316715" ,
"255415" ,
"310215" ,
"271415" ,
"248515" ,
"286715" ,
"240515" ,
"330915" ,
"259215" ,
"254315" ,
"251315" ,
"246015" ,
"257715" ,
"289815" ,
"328615" ,
"235415" ,
"310315" ,
"297815" ,
"283115" ,
"315015" ,
"245315" ,
"250115" ,
"297715" ,
"308015" ,
"255015" ,
"316415" ,
"311015" ,
"315815" ,
"312815" ,
"257315" ,
"253015" ,
"315115" ,
"325215" ,
"314515" ,
"321515" ,
"277415" ,
"238815" ,
"232915" ,
"281715" ,
"279715" ,
"292615" ,
"319715" ,
"297515" ,
"299315" ,
"306315" ,
"292415" ,
"294815" ,
"318515" ,
"286915" ,
"251815" ,
"253215" ,
"253415" ,
"332215" ,
"276715" ,
"329215" ,
"283415" ,
"319615" ,
"322515" ,
"320415" ,
"240615" ,
"286515" ,
"321815" ,
"304915" ,
"247015" ,
"326615" ,
"293315" ,
"301615" ,
"313515" ,
"323715" ,
"329115" ,
"236415" ,
"307615" ,
"273215" ,
"281515" ,
"307315" ,
"318115" ,
"281415" ,
"321415" ,
"279915" ,
"277215" ,
"330315" ,
"274415" ,
"307815" ,
"321315" ,
"311315" ,
"235815" ,
"322915" ,
"274615" ,
"282415" ,
"234515" ,
"281115" ,
"248715" ,
"250715" ,
"271915" ,
"237415" ,
"303415" ,
"236215" ,
"297915" ,
"274115" ,
"907807" ,
"905607" ,
"903907" ,
"906407" ,
"900807" ,
"908607" ,
"913907" ,
"910307" ,
"913807" ,
"912807" ,
"904207" ,
"910607" ,
"905207" ,
"912507" ,
"909607" ,
"916207" ,
"905507" ,
"911907" ,
"913407" ,
"901307" ,
"910907" ,
"901807" ,
"906107" ,
"909707" ,
"916007" ,
"915307" ,
"914207" ,
"901007" ,
"911407" ,
"903207" ,
"912407" ,
"905007" ,
"902307" ,
"902807" ,
"913507" ,
"911707" ,
"901607" ,
"908307" ,
"910007" ,
"912907" ,
"909007" ,
"901207" ,
"914107" ,
"638105" ,
"629505" ,
"629105" ,
"628705" ,
"639805" ,
"629005" ,
"628405" ,
"610705" ,
"627705" ,
"609705" ,
"622705" ,
"635405" ,
"639205" ,
"621405" ,
"629705" ,
"627605" ,
"635905" ,
"611005" ,
"610205" ,
"627005" ,
"630505" ,
"632505" ,
"619605" ,
"614805" ,
"633505" ,
"613405" ,
"609605" ,
"616105" ,
"620305" ,
"630405" ,
"614605" ,
"609005" ,
"624405" ,
"620105" ,
"620505" ,
"616905" ,
"638305" ,
"608905" ,
"625705" ,
"611105" ,
"619805" ,
"623005" ,
"631205" ,
"636105" ,
"612105" ,
"633305" ,
"626305" ,
"616005" ,
"626105" ,
"639005" ,
"614005" ,
"612305" ,
"634905" ,
"620605" ,
"617605" ,
"636205" ,
"632405" ,
"619405" ,
"626805" ,
"626205" ,
"633705" ,
"618505" ,
"620205" ,
"621005" ,
"626005" ,
"613505" ,
"636005" ,
"632605" ,
"624505" ,
"622605" ,
"637405" ,
"610505" ,
"632005" ,
"619305" ,
"624905" ,
"619105" ,
"614205" ,
"639305" ,
"630805" ,
"616805" ,
"609305" ,
"637905" ,
"616305" ,
"627805" ,
"636605" ,
"622305" ,
"631705" ,
"629605" ,
"618005" ,
"615605" ,
"626605" ,
"620805" ,
"619205" ,
"626505" ,
"637505" ,
"630605" ,
"621805" ,
"637805" ,
"638505" ,
"622805" ,
"629405" ,
"631805" ,
"632705" ,
"633805" ,
"620005" ,
"617705" ,
"615205" ,
"632305" ,
"629205" ,
"627505" ,
"636705" ,
"639105" ,
"616405" ,
"635705" ,
"623905" ,
"618305" ,
"617805" ,
"615005" ,
"126913" ,
"128913" ,
"137413" ,
"127413" ,
"128313" ,
"129113" ,
"132613" ,
"135313" ,
"133113" ,
"131413" ,
"135713" ,
"135013" ,
"134613" ,
"128813" ,
"128013" ,
"133813" ,
"138613" ,
"132713" ,
"135413" ,
"127913" ,
"137213" ,
"133913" ,
"129313" ,
"129913" ,
"129513" ,
"127713" ,
"127313" ,
"133713" ,
"135513" ,
"134813" ,
"132813" ,
"131013" ,
"130813" ,
"136013" ,
"138713" ,
"137013" ,
"129813" ,
"127513" ,
"132913" ,
"137513" ,
"133213" ,
"130213" ,
"129013" ,
"135813" ,
"134913" ,
"132313" ,
"136713" ,
"132213" ,
"133013" ,
"136413" ,
"133613" ,
"134313" ,
"126813" ,
"127213" ,
"134013" ,
"131613" ,
"131513" ,
"137113" ,
"128513" ,
"134213" ,
"135213" ,
"127113" ,
"127013" ,
"134113" ,
"131713" ,
"136513" ,
"136313" ,
"129413" ,
"138313" ,
"138813" ,
"131813" ,
"138113" ,
"132413" ,
"132113" ,
"126713" ,
"132513" ,
"629911" ,
"639811" ,
"630811" ,
"628711" ,
"630511" ,
"635911" ,
"637011" ,
"639611" ,
"625211" ,
"641411" ,
"622911" ,
"636511" ,
"627411" ,
"635111" ,
"633611" ,
"623811" ,
"641211" ,
"623411" ,
"628911" ,
"625311" ,
"638411" ,
"632511" ,
"632011" ,
"626711" ,
"630711" ,
"634411" ,
"632711" ,
"624511" ,
"633911" ,
"637311" ,
"627611" ,
"637111" ,
"639311" ,
"628311" ,
"635011" ,
"622311" ,
"626911" ,
"625911" ,
"630911" ,
"636211" ,
"633211" ,
"637911" ,
"623511" ,
"626211" ,
"639111" ,
"640611" ,
"631711" ,
"625711" ,
"640111" ,
"641111" ,
"632411" ,
"633011" ,
"638211" ,
"624111" ,
"638311" ,
"630411" ,
"640811" ,
"634711" ,
"631411" ,
"629511" ,
"633711" ,
"635211" ,
"626811" ,
"626511" ,
"633811" ,
"640511" ,
"628011" ,
"627111" ,
"634111" ,
"630111" ,
"639211" ,
"631311" ,
"624011" ,
"635811" ,
"636611" ,
"627811" ,
"629011" ,
"635411" ,
"624411" ,
"634511" ,
"637711" ,
"640411" ,
"633411" ,
"633311" ,
"639911" ,
"629211" ,
"640311" ,
"222409" ,
"225509" ,
"215809" ,
"230309" ,
"215209" ,
"225709" ,
"216009" ,
"223909" ,
"222709" ,
"221309" ,
"216709" ,
"226409" ,
"231509" ,
"232809" ,
"230109" ,
"232009" ,
"216809" ,
"215309" ,
"217709" ,
"220409" ,
"223409" ,
"221509" ,
"228609" ,
"218809" ,
"231309" ,
"228109" ,
"222609" ,
"222509" ,
"227809" ,
"217609" ,
"225809" ,
"225609" ,
"227409" ,
"230509" ,
"228509" ,
"229509" ,
"227509" ,
"218309" ,
"216509" ,
"221609" ,
"219009" ,
"231809" ,
"223509" ,
"224009" ,
"215709" ,
"220709" ,
"226609" ,
"226509" ,
"229909" ,
"231909" ,
"157814" ,
"160414" ,
"152014" ,
"151914" ,
"159514" ,
"169814" ,
"156914" ,
"154014" ,
"153814" ,
"153014" ,
"150414" ,
"156814" ,
"150214" ,
"159414" ,
"152614" ,
"152314" ,
"152214" ,
"156514" ,
"146314" ,
"151814" ,
"154614" ,
"156314" ,
"150314" ,
"156714" ,
"151014" ,
"157914" ,
"158814" ,
"158014" ,
"157014" ,
"156114" ,
"154814" ,
"156414" ,
"339703" ,
"335803" ,
"337703" ,
"355403" ,
"338303" ,
"321403" ,
"343503" ,
"340403" ,
"320103" ,
"358803" ,
"343303" ,
"355003" ,
"329703" ,
"328103" ,
"333703" ,
"360603" ,
"329603" ,
"360903" ,
"333403" ,
"334403" ,
"320703" ,
"341503" ,
"343003" ,
"326203" ,
"353703" ,
"333503" ,
"320203" ,
"326403" ,
"359603" ,
"358103" ,
"342403" ,
"341703" ,
"338503" ,
"351903" ,
"345303" ,
"347003" ,
"331603" ,
"352603" ,
"328903" ,
"326103" ,
"348703" ,
"333203" ,
"343603" ,
"342803" ,
"331703" ,
"326603" ,
"335903" ,
"341403" ,
"339503" ,
"333103" ,
"327303" ,
"357203" ,
"337303" ,
"336003" ,
"350503" ,
"351503" ,
"358403" ,
"330803" ,
"338603" ,
"346403" ,
"323203" ,
"329403" ,
"343703" ,
"340703" ,
"328703" ,
"316803" ,
"325903" ,
"356703" ,
"326703" ,
"345603" ,
"338003" ,
"359303" ,
"349203" ,
"341103" ,
"345003" ,
"330603" ,
"322003" ,
"352103" ,
"352203" ,
"321103" ,
"358203" ,
"352303" ,
"338903" ,
"316903" ,
"329803" ,
"360803" ,
"346203" ,
"329503" ,
"325003" ,
"316503" ,
"340103" ,
"318003" ,
"338803" ,
"340803" ,
"330403" ,
"345903" ,
"334903" ,
"345803" ,
"332703" ,
"327503" ,
"354603" ,
"315403" ,
"315103" ,
"332503" ,
"318403" ,
"343203" ,
"344203" ,
"343103" ,
"325303" ,
"347303" ,
"355303" ,
"338703" ,
"359903" ,
"359403" ,
"333303" ,
"350803" ,
"353103" ,
"349403" ,
"339603" ,
"338103" ,
"349903" ,
"340203" ,
"323003" ,
"354203" ,
"347203" ,
"351103" ,
"343403" ,
"332603" ,
"316703" ,
"330503" ,
"359103" ,
"340303" ,
"334203" ,
"344603" ,
"356603" ,
"356403" ,
"319903" ,
"319003" ,
"354703" ,
"318803" ,
"350603" ,
"352503" ,
"321803" ,
"315903" ,
"328403" ,
"318603" ,
"351003" ,
"341803" ,
"343903" ,
"330303" ,
"327803" ,
"339903" ,
"340003" ,
"322303" ,
"332303" ,
"335403" ,
"352903" ,
"324403" ,
"324503" ,
"343803" ,
"322403" ,
"330703" ,
"325703" ,
"345203" ,
"348503" ,
"326003" ,
"353503" ,
"326803" ,
"353303" ,
"328803" ,
"337803" ,
"329903" ,
"334703" ,
"330903" ,
"354403" ,
"323603" ,
"315803" ,
"359203" ,
"353403" ,
"357003" ,
"344303" ,
"316203" ,
"353903" ,
"348803" ,
"347703" ,
"337103" ,
"336103" ,
"332903" ,
"317003" ,
"322603" ,
"324803" ,
"321303" ,
"359703" ,
"346903" ,
"328503" ,
"353003" ,
"317203" ,
"331203" ,
"350203" ,
"339803" ,
"338203" ,
"348403" ,
"345503" ,
"357703" ,
"342503" ,
"356903" ,
"337403" ,
"358003" ,
"336903" ,
"354503" ,
"335703" ,
"321903" ,
"318903" ,
"350003" ,
"320403" ,
"339003" ,
"321603" ,
"330103" ,
"356003" ,
"344103" ,
"324603" ,
"318303" ,
"325103" ,
"320803" ,
"318203" ,
"350703" ,
"354303" ,
"330203" ,
"320603" ,
"315503" ,
"351303" ,
"344803" ,
"334503" ,
"347803" ,
"346803" ,
"360303" ,
"319703" ,
"354003" ,
"331303" ,
"349103" ,
"358703" ,
"321003" ,
"357803" ,
"351403" ,
"317503" ,
"323803" ,
"322203" ,
"327003" ,
"333603" ,
"361203" ,
"349003" ,
"346603" ,
"346503" ,
"338403" ,
"316603" ,
"335203" ,
"323503" ,
"317803" ,
"315603" ,
"321703" ,
"332203" ,
"334603" ,
"317903" ,
"331503" ,
"327103" ,
"355803" ,
"352003" ,
"341603" ,
"331403" ,
"319303" ,
"342703" ,
"339303" ,
"329003" ,
"344003" ,
"340903" ,
"344503" ,
"319203" ,
"347903" ,
"320303" ,
"330003" ,
"335303" ,
"327903" ,
"318103" ,
"360003" ,
"357103" ,
"350903" ,
"342603" ,
"337003" ,
"335003" ,
"356203" ,
"349703" ,
"315703" ,
"327703" ,
"317603" ,
"351603" ,
"360403" ,
"357303" ,
"320003" ,
"323403" ,
"357503" ,
"322803" ,
"332403" ,
"334103" ,
"360103" ,
"360503" ,
"340603" ,
"323903" ,
"357903" ,
"356503" ,
"359503" ,
"342303" ,
"331103" ,
"318703" ,
"327603" ,
"326903" ,
"348903" ,
"322703" ,
"345703" ,
"332803" ,
"350403" ,
"335103" ,
"358903" ,
"342903" ,
"331003" ,
"351703" ,
"356303" ,
"353203" ,
"319603" ,
"355703" ,
"317103" ,
"352803" ,
"320503" ,
"354103" ,
"317403" ,
"328203" ,
"326503" ,
"319803" ,
"336203" ,
"323303" ,
"346303" ,
"342203" ,
"336503" ,
"316103" ,
"319503" ,
"323103" ,
"331803" ,
"328003" ,
"360703" ,
"355503" ,
"331903" ,
"357403" ,
"335503" ,
"334003" ,
"315303" ,
"337503" ,
"316003" ,
"350103" ,
"332103" ,
"319403" ,
"342103" ,
"358303" ,
"322103" ,
"329303" ,
"327403" ,
"319103" ,
"337603" ,
"348003" ,
"346003" ,
"341003" ,
"339403" ,
"360203" ,
"315203" ,
"329103" ,
"359003" ,
"329203" ,
"324703" ,
"348303" ,
"347103" ,
"326303" ,
"320903" ,
"361003" ,
"325403" ,
"317303" ,
"347603" ,
"324303" ,
"323703" ,
"324203" ,
"351203" ,
"358603" ,
"353603" ,
"348603" ,
"356803" ,
"321203" ,
"339203" ,
"317703" ,
"349803" ,
"347503" ,
"357603" ,
"349503" ,
"355103" ,
"322503" ,
"333803" ,
"352703" ,
"345403" ,
"340503" ,
"324903" ,
"346703" ,
"339103" ,
"341203" ,
"354803" ,
"352403" ,
"325503" ,
"337203" ,
"344903" ,
"321503" ,
"333003" ,
"358503" ,
"316303" ,
"336303" ,
"341303" ,
"353803" ,
"334803" ,
"335603" ,
"327203" ,
"351803" ,
"341903" ,
"325803" ,
"355903" ,
"342003" ,
]

### load the pedigree and phenotype

In [18]:
pedigree = Pedigree(pedigree_file='/mnt/bbg/AIL_reseq/useful_info/pedigree/AIL_pedigree_20190826.tsv')

In [19]:
ph = pd.read_csv('../data/AIL_phenotypes20190827.tsv', sep='\t')

In [20]:
ph.index = ph.id.astype(str)

In [21]:
gt_ids2 = list(set.intersection(set(ph.index), set(gt_ids)))

In [22]:
import copy

In [23]:
ph2 = copy.deepcopy(ph.loc[gt_ids2])

In [24]:
len(ph2.columns)

74

In [25]:
ph2.columns

Index(['id', 'Unnamed: 0', 'ABDFAT', 'BREAST_L_M', 'BREAST_R_M', 'BURSA',
       'BW0', 'BW10', 'BW2', 'BW4', 'BW6', 'BW8', 'BW_HATCH', 'B_PROT', 'CHOL',
       'CHOLESTEROL', 'DAM', 'DOWN', 'FAT', 'GENERATION', 'GLG', 'GLGO1',
       'GLGO1S', 'GLUCOM', 'GLUCOSE', 'GLUL', 'GLUO1', 'GLUO1L', 'GR02',
       'GR24', 'GR46', 'GR68', 'GR810', 'HATCH', 'ID', 'IG1CO2S', 'IGF1C',
       'IGF1CO1', 'IGF1CO2', 'IGF1COS', 'INS', 'INSO1', 'INSO1S', 'INSS',
       'INSULIN', 'L_BMUS', 'L_LUNG', 'L_SHK', 'MELANIN', 'PANCREAS', 'PCVM',
       'R_BMUS', 'R_LUNG', 'R_SHK', 'SEX', 'SHANK_L_D', 'SHANK_L_L',
       'SHANK_L_W', 'SHANK_R_D', 'SHANK_R_L', 'SHANK_R_W', 'SIRE', 'SPLN',
       'SRBC', 'S_BMUS', 'S_LUNG', 'S_SHK', 'TG2', 'TG2O1', 'TG2O1S',
       'left_shank_with_toes', 'out', 'NOTES', 'BW8_norm_by_gen'],
      dtype='object')

In [26]:
bw8 = ph2[['id','BW8','BW8_norm_by_gen', 'GENERATION', 'SEX', 'DAM']]

In [ ]:
# fill in  parental information

In [27]:
missing_dam = list(bw8.loc[bw8.DAM.isna()]['id'].astype(str))

In [28]:
len(list(bw8.DAM.astype(int).astype(str)))

3133

In [29]:
missing_dam # no damn dam missing :) 

[]

In [30]:
missing_sex = list(bw8.loc[bw8.SEX.isna()]['id'].astype(str))

In [31]:
missing_sex # just one, great

['619805']

In [33]:
pedigree.get_offspring(sample=missing_sex[0]) # unfortunately no offspring, so i cant check the records if it was a dam or sire 

[]

In [34]:
ph.loc[['619805']].SEX

id
619805   NaN
Name: SEX, dtype: float64

In [32]:
bw8.SEX # but its the wrong format. rqtl wants females as 0, males as 1, here females are coded as 2

id
639911    1.0
300915    1.0
94602     2.0
327215    2.0
286815    1.0
         ... 
530608    1.0
67702     2.0
90202     1.0
317715    2.0
282815    1.0
Name: SEX, Length: 3133, dtype: float64

In [35]:
bw8.replace(2.0, 0.0, inplace=True)

/home/tilman/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [36]:
bw8.replace(np.nan, 'NA', inplace=True) # for R/qtl

In [38]:
bw8.GENERATION = ['F'+str(i) for i in bw8.GENERATION] # make sure GENERATION is a factor, not numeric


/home/tilman/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [41]:
bw8.DAM = ['DAM_'+str(i) for i in bw8.DAM] # make sure DAM is a factor, not numeric

/home/tilman/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [42]:
bw8.to_csv('20200826_intersection_phenotypes.csv', sep=',')

4485

3135

In [87]:
fil2.to_csv('20200708_intersection_normbw8.csv', sep=',')